<a href="https://colab.research.google.com/github/bokutachi256/gisday2021/blob/main/2_%E3%82%A8%E3%83%BC%E3%82%B8%E3%82%A7%E3%83%B3%E3%83%88%E3%81%8C%E3%81%84%E3%81%AA%E3%81%84%E3%82%BB%E3%83%AB%E3%81%AB%E3%81%AE%E3%81%BF%E7%A7%BB%E5%8B%95%E3%81%99%E3%82%8B%5BGISDAY2021%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Pythonで作るマルチエージェントシミュレーション
* 東京都立大学 都市環境学部 地理環境学科 中山大地
* 2021年12月4日 東京都立大学 南大沢キャンパス（オンライン開催）
* このテキストのURL [https://github.com/bokutachi256/gisday2021](https://github.com/bokutachi256/gisday2021)
* Google ColaboratoryのURL [https://colab.research.google.com/](https://colab.research.google.com/)
* MESAのURL [https://mesa.readthedocs.io/](https://mesa.readthedocs.io/)

# この章でやること

この章では避難者エージェントの行動ルールを改良します．
具体的には避難者エージェントは他の避難者エージェントがいないセルにのみ移動するようにします．

# Mesaのインストール

In [ ]:
%pip install mesa

# 必要ライブラリのインポート

In [ ]:
%matplotlib notebook

import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
from matplotlib import animation, rc
from matplotlib.animation import ArtistAnimation
from IPython.display import HTML

from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.space import SingleGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector

# 当初のモデルの改変

## スペースのモデルを改造して避難者エージェントのスタート地点を作る





前の章で作成したスペースは以下のようになっていました．

* スペースはMultiGridを使う
* エージェントの初期位置はランダムに決定する
* ゴールは`goal_x`と`goal_y`とする
* データコレクターには以下の情報を格納する
 * model_reporter：ゴールした人数の累計，Stepごとのゴールした人数
 * agent_reporter：エージェントの座標とゴールしたかどうかの状態を保持

```python
class EvacModel(Model):
  def __init__(self, N, width, height, goal_x, goal_y):
    # 避難者エージェントの数をNに設定する
    self.num_agents = N
    # MultiGridは同一座標に複数エージェントを配置できるスペース
    self.grid = MultiGrid(width, height, torus=False)
    # ゴール地点の座標を設定する（goal_xとgoal_y)
    self.goal = (goal_x, goal_y)
    # 避難完了者をカウントする（0で初期化）
    self.evac_comp = 0
    # EvacAgentの実行順序をランダムに設定する
    self.schedule = RandomActivation(self)

    # 避難者エージェントの作成．num_agentsの数だけエージェントを作成する
    for i in range(self.num_agents):
      a = EvacAgent(i, self)
      self.schedule.add(a)
      # エージェントの初期位置をランダムに決定する
      x = self.random.randrange(self.grid.width)
      y = self.random.randrange(self.grid.height)
      # エージェントを配置する
      self.grid.place_agent(a, (x, y))

      # データコレクターの設定
      self.datacollector = DataCollector(
        model_reporters={"Goal": compute_goal, "evacuator": goal_pop},
        agent_reporters={"Pos": "pos", "Goal": "mygoal"})
      
    self.datacollector.collect(self)
      
  def step(self):
    model.evac_comp = 0
    self.schedule.step()
    self.datacollector.collect(self)
```

このモデルを改良して，避難者エージェントのスタート地点を固定します．
スペースのコンストラクタに避難者エージェントのスタート地点を表す引数を追加し，
その座標に避難者エージェントを配置します．

コンストラクタは以下のようになります．

```python
  def __init__(self, N, width, height, start_x, start_y, goal_x, goal_y):
```

エージェントの配置部分は以下のように変更します．

```python
      self.grid.place_agent(a, (start_x, start_y))
```

まとめるとこのようになります．

In [ ]:
class EvacModel(Model):
  def __init__(self, N, width, height, start_x, start_y, goal_x, goal_y):
    # 避難者エージェントの数をNに設定する
    self.num_agents = N
    # MultiGridは同一座標に複数エージェントを配置できるスペース
    self.grid = MultiGrid(width, height, torus=False)
    # ゴール地点の座標を設定する（goal_xとgoal_y)
    self.goal = (goal_x, goal_y)
    # 避難完了者をカウントする（0で初期化）
    self.evac_comp = 0
    # EvacAgentの実行順序をランダムに設定する
    self.schedule = RandomActivation(self)

    # 避難者エージェントの作成．num_agentsの数だけエージェントを作成する
    for i in range(self.num_agents):
      a = EvacAgent(i, self)
      self.schedule.add(a)
      # エージェントを配置する
      self.grid.place_agent(a, (start_x, start_y))

      # データコレクターの設定
      self.datacollector = DataCollector(
        model_reporters={"Goal": compute_goal, "evacuator": goal_pop},
        agent_reporters={"Pos": "pos", "Goal": "mygoal"})
      
    self.datacollector.collect(self)
      
  def step(self):
    model.evac_comp = 0
    self.schedule.step()
    self.datacollector.collect(self)


## ゴール人数の累計を求める関数

こちらは変更ありません．

In [ ]:
# そのステップでゴールした人数を求める関数
def goal_pop(model):
  return(model.evac_comp)

# ゴールした人数の累計を求める関数
def compute_goal(model):
  agent_goal = sum([agent.mygoal for agent in model.schedule.agents])
  return (agent_goal)

## 避難者エージェントの定義



これも変更ありません．

In [ ]:
class EvacAgent(Agent):

  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    self.mygoal = 0

  def move(self):
    possible_steps = self.model.grid.get_neighborhood(
      pos=self.pos,
      moore = False,
      include_center = False)
    new_position = self.random.choice(possible_steps)
    self.model.grid.move_agent(self, new_position)

  # ゴールに到達したか判定する関数
  def if_goal(self):
    if self.pos == self.model.goal:
      self.mygoal += 1
      self.model.evac_comp += 1

  def step(self):
    if self.mygoal == 0:
      self.move()
      self.if_goal()


## シミュレーションを実行する

スタート地点を表す`start_x`と`start_y`を作り，
スペースのインスタンス生成に加えます．

In [ ]:
start_x = 4
start_y = 4
goal_x = 0
goal_y = 0

model = EvacModel(10, 5, 5, start_x, start_y, goal_x, goal_y)

while min([agent.mygoal for agent in model.schedule.agents]) < 1:
  model.step()

## 計算結果のアニメーション

計算結果のアニメーションを作成します．

In [ ]:
fig = plt.figure(figsize = (5, 5))
ax = fig.add_subplot(111)

# ゴールをプロット
plt.text(goal_x, goal_y, 'Goal', size = 14, ha = 'center')
plt.scatter(goal_x, goal_y, s = 800, marker = "s")

ims = []
step = 3

# 0 Stepめのエージェントの座標を取得する
agent_loc = model.datacollector.get_agent_vars_dataframe().xs(0, level='Step')['Pos']
x0, y0 = [a for a, b in agent_loc], [b for a, b in agent_loc]

for i in tqdm(range(1, 100)): 
  agent_loc = model.datacollector.get_agent_vars_dataframe().xs(i, level='Step')['Pos']
  x, y = [a for a, b in agent_loc], [b for a, b in agent_loc]
  dx = (np.array(x) - np.array(x0)) / step
  dy = (np.array(y) - np.array(y0)) / step
  for u in range(step):
    im = ax.scatter(x0 + (dx * u), y0 + (dy * u), s=1000, c="red", marker="*")
    # グラフをリストに加える
    ims.append([im])
  x0 = x
  y0 = y

ax.set_title('evacuation')
ax.set_xlabel('x')
ax.set_ylabel('y')

# ArtistAnimationにfigオブジェクトとimsを代入してアニメーションを作成
anim = animation.ArtistAnimation(fig, ims, interval = 50)

# Google Colaboratoryの場合必要
rc('animation', html='jshtml')
plt.close()
anim

スタート地点から避難者エージェントが移動していくのがわかります．

## このシミュレーションの欠点

このシミュレーションにおける避難者エージェントは上下左右にランダムで移動しています．
つまり移動先に他のエージェントがいても無視していることになります．
そこでプログラムを改造し，他の避難者エージェントがいないセルにのみ移動するようにします．
これは他のエージェントの状態によって自分が影響を受けることになり，
マルチエージェントシミュレーションの意義が出ることになります．

# エージェントがいないセルを検索するアルゴリズム

現状では避難者エージェントの`move`メソッドは以下のようになっています．

```python
  def move(self):
    possible_steps = self.model.grid.get_neighborhood(
      pos=self.pos,
      moore = False,
      include_center = False)
    new_position = self.random.choice(possible_steps)
    self.model.grid.move_agent(self, new_position)
```

`model.grid.get_neighborhood`メソッドを使って周囲のセルを検索し，
取得されたセルのリストから`random.choice`で移動方向を決定しています．

ここで`model.grid.get_neighborhood`で取得された周囲セルのリスト`possible_step`で示された座標に避難者エージェントが含まれていた場合，そのセルを移動候補から削除すれば
他のエージェントがいないセルにのみ移動することができます．

`model.grid.get_cell_list_contents`メソッドは指定された座標に配置されているエージェントのリストを取得するメソッドです．
指定された座標にエージェントがない場合には戻り値は空のリストになるため，`len(model.grid.get_cell_list_contents)`は0になります．
これを使って，避難者エージェントの上下左右で他のエージェントがいないセルを検索するには以下のようにします．

```python
    possible_steps2 = [a for a in possible_steps if len(self.model.grid.get_cell_list_contents(a)) < 1]
```

`possible_steps2`には上下左右のセルのうち，避難者エージェントがいないセルの位置のみがリストとして取得できます．
次に`possible_step2`からランダムで移動方向を選べば良いのですが，
場合によっては上下左右のセルがすべて埋まっているかもしれません．
その場合は避難者エージェントは移動しないようにします．

これは以下のように書けます．

```python
    if len(possible_steps2) > 0:
      new_position = self.random.choice(possible_steps2)
    else:
      new_position = self.pos
    self.model.grid.move_agent(self, new_position)
```

以上をまとめると避難者エージェントはこのようになります．

In [ ]:
class EvacAgent(Agent):

  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    self.mygoal = 0
    self.wealth = 0

  def move(self):
    possible_steps = self.model.grid.get_neighborhood(
      self.pos,
      moore = False,
      include_center = False)

    # 周囲にエージェントがいないセルがあるか検索する
    possible_steps2 = [a for a in possible_steps if len(self.model.grid.get_cell_list_contents(a)) < 1]

    if len(possible_steps2) > 0:
      new_position = self.random.choice(possible_steps2)
    else:
      new_position = self.pos
    self.model.grid.move_agent(self, new_position)
  
  # ゴールに到達したか判定する関数
  def if_goal(self):
    if self.pos == self.model.goal:
      self.mygoal += 1
      self.model.evac_comp += 1

  def step(self):
    if self.mygoal == 0:
      self.move()
      self.if_goal()


## モデルを実行する

とりあえず100回実行してみましょう

In [ ]:
start_x = 4
start_y = 4
goal_x = 0
goal_y = 0

model = EvacModel(20, 5, 5, start_x, start_y, goal_x, goal_y)

# とりあえず100回実行する

for i in range(100):
  model.step()

## 計算結果をアニメーションとグラフで検討する

In [ ]:
fig = plt.figure(figsize = (5, 5))
ax = fig.add_subplot(111)

# ゴールをプロット
plt.text(goal_x, goal_y, 'Goal', size = 14, ha = 'center')
plt.scatter(goal_x, goal_y, s = 1000, marker = "s")

ims = []
step = 3

# 0 Stepめのエージェントの座標を取得する
agent_loc = model.datacollector.get_agent_vars_dataframe().xs(0, level='Step')['Pos']
x0, y0 = [a for a, b in agent_loc], [b for a, b in agent_loc]

for i in tqdm(range(1, 100)): 
  agent_loc = model.datacollector.get_agent_vars_dataframe().xs(i, level='Step')['Pos']
  x, y = [a for a, b in agent_loc], [b for a, b in agent_loc]
  dx = (np.array(x) - np.array(x0)) / step
  dy = (np.array(y) - np.array(y0)) / step
  for u in range(step):
    im = ax.scatter(x0 + (dx * u), y0 + (dy * u), s=600, c="red", marker="*")
    # グラフをリストに加える
    ims.append([im])
  x0 = x
  y0 = y

ax.set_title('evacuation')
ax.set_xlabel('x')
ax.set_ylabel('y')

# ArtistAnimationにfigオブジェクトとimsを代入してアニメーションを作成
anim = animation.ArtistAnimation(fig, ims, interval = 50)

# Google Colaboratoryの場合必要
rc('animation', html='jshtml')
plt.close()
anim

確かに避難者エージェントは重ならないようになっていますが，
避難者エージェントの振る舞いが何かおかしいです．
ゴールした人数をグラフ化して確認してみましょう

In [ ]:
%matplotlib inline

fig = plt.figure()
ax = fig.add_subplot(111)

goal = model.datacollector.get_model_vars_dataframe()


ax.plot(goal)
plt.show

最初に一人がゴールした後は，誰もゴールしていないようです．

避難者エージェントはゴールするとゴール地点に止まるため，
他の避難者エージェントがゴールできなくなっています．

これでは困るのでゴールしたエージェントは削除するようにします．


# ゴール後のエージェントを削除する

## エージェントの削除方法

エージェントは`grid.remove_aget`メソッドで削除できます．
避難エージェントがゴールしたら削除します．

避難者エージェントの`if_goal`メソッドに避難者エージェントの削除を追加します．
`remove_agent`メソッドの引数は削除したいエージェントのインスタンスです．
ここでは自分自身を削除するため`self`とします．
```python
  def if_goal(self):
    if self.pos == self.model.goal:
      self.mygoal += 1
      self.model.evac_comp += 1
      # ゴールしたらエージェントを削除する
      self.model.grid.remove_agent(self)
```

まとめると避難者エージェント`EvacAgent`は以下のようになります．
実行してみましょう．

In [ ]:
class EvacAgent(Agent):

  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    self.mygoal = 0
    self.wealth = 0

  def move(self):
    possible_steps = self.model.grid.get_neighborhood(
      self.pos,
      moore = False,
      include_center = False)

    # 周囲に移動可能なセルがあるか検索する
    possible_steps2 = [a for a in possible_steps if len(self.model.grid.get_cell_list_contents(a)) < 1]

    if len(possible_steps2) > 0:
      new_position = self.random.choice(possible_steps2)
    else:
      new_position = self.pos
    self.model.grid.move_agent(self, new_position)
  
  # ゴールに到達したか判定する関数
  def if_goal(self):
    if self.pos == self.model.goal:
      self.mygoal += 1
      self.model.evac_comp += 1
      # ゴールしたらエージェントを削除する
      self.model.grid.remove_agent(self)


  def step(self):
    if self.mygoal == 0:
      self.move()
      self.if_goal()


## モデルを再実行する

避難者エージェントを改変したので，モデルを再実行します．

In [ ]:
start_x = 4
start_y = 4
goal_x = 0
goal_y = 0

model = EvacModel(20, 5, 5, start_x, start_y, goal_x, goal_y)

# とりあえず100回実行する
for i in range(100):
  model.step()

## データコレクターから実行結果を検討する

避難者エージェントのデータコレクターを確認します．

In [ ]:
model.datacollector.get_agent_vars_dataframe()

Pos  Goal
Step AgentID              
0    0        (4, 4)     0
     1        (4, 4)     0
     2        (4, 4)     0
     3        (4, 4)     0
     4        (4, 4)     0
...              ...   ...
273  15         None     1
     16         None     1
     17         None     1
     18         None     1
     19         None     1

[5480 rows x 2 columns]

最終ステップの`Pos`が`None`になっています．
削除されたエージェントは削除後に位置を示すプロパティ`pos`が`None`になります．
このままではアニメーション作成時にエラーが出てしまうため，`None`をゴールの座標(0, 0)に置き換えます．

確認のため第99ステップ目の避難者エージェントの座標を見てみましょう．


In [ ]:
model.datacollector.get_agent_vars_dataframe().xs(99, level='Step')['Pos']

AgentID
0       None
1       None
2       None
3     (3, 0)
4       None
5       None
6       None
7       None
8       None
9       None
10    (4, 2)
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
Name: Pos, dtype: object

いくつか`None`が含まれています，
これを`map`とlambda式を用いて(0, 0)に置き換えます．

以下のようになります．

In [ ]:
model.datacollector.get_agent_vars_dataframe().xs(99, level='Step')['Pos'].map(lambda x: (0, 0) if x is None else x)

AgentID
0     (0, 0)
1     (0, 0)
2     (0, 0)
3     (3, 0)
4     (0, 0)
5     (0, 0)
6     (0, 0)
7     (0, 0)
8     (0, 0)
9     (0, 0)
10    (4, 2)
11    (0, 0)
12    (0, 0)
13    (0, 0)
14    (0, 0)
15    (0, 0)
16    (0, 0)
17    (0, 0)
18    (0, 0)
19    (0, 0)
Name: Pos, dtype: object

避難エージェントのアニメーションを作成するときにこの置き換えをすれば良いでしょう．

## 計算結果をアニメーション表示する

ゴール後の避難者エージェント削除に対応したアニメーション作成は以下のようになります．
削除されたエージェントの座標`None`を`(0, 0)`に置き換えています．

In [ ]:
fig = plt.figure(figsize = (5, 5))
ax = fig.add_subplot(111)

# ゴールをプロット
plt.text(goal_x, goal_y, 'Goal', size = 14, ha = 'center')
plt.scatter(goal_x, goal_y, s = 800, marker = "s")

ims = []
step = 3

# 0 Stepめのエージェントの座標を取得する
agent_loc = model.datacollector.get_agent_vars_dataframe().xs(0, level='Step')['Pos'].map(lambda x: (0, 0) if x is None else x)
x0, y0 = [a for a, b in agent_loc], [b for a, b in agent_loc]

for i in tqdm(range(1, 100)):
  agent_loc = model.datacollector.get_agent_vars_dataframe().xs(i, level='Step')['Pos'].map(lambda x: (0, 0) if x is None else x)
  x, y = [a for a, b in agent_loc], [b for a, b in agent_loc]
  dx = (np.array(x) - np.array(x0)) / step
  dy = (np.array(y) - np.array(y0)) / step
  for u in range(step):
    im = ax.scatter(x0 + (dx * u), y0 + (dy * u), s=1000, c="red", marker="*")
    # グラフをリストに加える
    ims.append([im])
  x0 = x
  y0 = y

ax.set_title('evacuation')
ax.set_xlabel('x')
ax.set_ylabel('y')

# ArtistAnimationにfigオブジェクトとimsを代入してアニメーションを作成
anim = animation.ArtistAnimation(fig, ims, interval = 50)

# Google Colaboratoryの場合必要
rc('animation', html='jshtml')
plt.close()
anim

## スペースの状態をグラフ化する

避難完了エージェントの推移グラフも正常になりました．

In [ ]:
%matplotlib inline

fig = plt.figure()
ax = fig.add_subplot(111)

goal = model.datacollector.get_model_vars_dataframe()


ax.plot(goal)
plt.show

# ここまでのまとめ

ここまでの内容をまとめました．

## スペースとエージェントを定義してモデルを実行する

In [ ]:
%pip install mesa
%matplotlib notebook

import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
from matplotlib import animation, rc
from matplotlib.animation import ArtistAnimation
from IPython.display import HTML

from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.space import SingleGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector

class EvacModel(Model):
  """A model with some number of agents."""
  # コンストラクタにgoal_xとgoal_yを追加した
  def __init__(self, N, width, height, start_x, start_y, goal_x, goal_y):
    # 避難者エージェントの数をNに設定する
    self.num_agents = N
    # MultiGridは同一座標に複数エージェントを配置できるスペース
    self.grid = MultiGrid(width, height, torus=False)

    # ゴール地点の座標を設定する（goal_xとgoal_y)
    self.goal = (goal_x, goal_y)
    # 避難完了者をカウントする（0で初期化）
    self.evac_comp = 0
    # MoneyAgentの実行順序をランダムに設定する
    self.schedule = RandomActivation(self)

    # 避難者エージェントの作成．num_agentsの数だけエージェントを作成する
    for i in range(self.num_agents):
      a = EvacAgent(i, self)
      self.schedule.add(a)
      # エージェントを配置する
      self.grid.place_agent(a, (start_x, start_y))

      # データコレクターの設定
      self.datacollector = DataCollector(
        model_reporters={"Goal": compute_goal, "evacuator": goal_pop},
        agent_reporters={"Pos": "pos", "Goal": "mygoal"})
      
    self.datacollector.collect(self)
      
  def step(self):
    model.evac_comp = 0
    self.schedule.step()
    self.datacollector.collect(self)

# そのステップでゴールした人数を求める関数
def goal_pop(model):
  return(model.evac_comp)

# ゴールした人数の累計を求める関数
def compute_goal(model):
  agent_goal = sum([agent.mygoal for agent in model.schedule.agents])
  return (agent_goal)

class EvacAgent(Agent):

  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    self.mygoal = 0
    self.wealth = 0

  def move(self):
    possible_steps = self.model.grid.get_neighborhood(
      self.pos,
      moore = False,
      include_center = False)

    # 周囲に移動可能なセルがあるか検索する
    possible_steps2 = [a for a in possible_steps if len(self.model.grid.get_cell_list_contents(a)) < 1]

    if len(possible_steps2) > 0:
      new_position = self.random.choice(possible_steps2)
    else:
      new_position = self.pos
    self.model.grid.move_agent(self, new_position)
  
  # ゴールに到達したか判定する関数
  def if_goal(self):
    if self.pos == self.model.goal:
      self.mygoal += 1
      self.model.evac_comp += 1
      # ゴールしたらエージェントを削除する
      self.model.grid.remove_agent(self)

  def step(self):
    if self.mygoal == 0:
      self.move()
      self.if_goal()

# モデルを実行する
start_x = 4
start_y = 4
goal_x = 0
goal_y = 0

model = EvacModel(20, 5, 5, start_x, start_y, goal_x, goal_y)

# 全員ゴールするまで実行する

while min([agent.mygoal for agent in model.schedule.agents]) < 1:
  model.step()

## 計算結果のアニメーション

In [ ]:
fig = plt.figure(figsize = (5, 5))
ax = fig.add_subplot(111)

# ゴールをプロット
plt.text(goal_x, goal_y, 'Goal', size = 14, ha = 'center')
plt.scatter(goal_x, goal_y, s = 800, marker = "s")

ims = []
step = 3

# 0 Stepめのエージェントの座標を取得する
agent_loc = model.datacollector.get_agent_vars_dataframe().xs(0, level='Step')['Pos'].map(lambda x: (0, 0) if x is None else x)
x0, y0 = [a for a, b in agent_loc], [b for a, b in agent_loc]

for i in tqdm(range(1, 100)):
  agent_loc = model.datacollector.get_agent_vars_dataframe().xs(i, level='Step')['Pos'].map(lambda x: (0, 0) if x is None else x)
  x, y = [a for a, b in agent_loc], [b for a, b in agent_loc]
  dx = (np.array(x) - np.array(x0)) / step
  dy = (np.array(y) - np.array(y0)) / step
  for u in range(step):
    im = ax.scatter(x0 + (dx * u), y0 + (dy * u), s=1000, c="red", marker="*")
    # グラフをリストに加える
    ims.append([im])
  x0 = x
  y0 = y

ax.set_title('evacuation')
ax.set_xlabel('x')
ax.set_ylabel('y')

# ArtistAnimationにfigオブジェクトとimsを代入してアニメーションを作成
anim = animation.ArtistAnimation(fig, ims, interval = 50)

# Google Colaboratoryの場合必要
rc('animation', html='jshtml')
plt.close()
anim

## スペースの状態をグラフ化する

In [ ]:
%matplotlib inline

fig = plt.figure()
ax = fig.add_subplot(111)

goal = model.datacollector.get_model_vars_dataframe()


ax.plot(goal)
plt.show